<a href="https://colab.research.google.com/github/akhilvydyula/Restaurant-Rating-Predict/blob/main/stacking_classifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing liberaries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from numpy import mean
from numpy import std
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import StackingClassifier
import re
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

importing dataset

In [16]:
dataset = pd.read_csv("/content/drive/MyDrive/Restaurant /zomato.csv",nrows=10000)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   url                          10000 non-null  object
 1   address                      10000 non-null  object
 2   name                         10000 non-null  object
 3   online_order                 10000 non-null  object
 4   book_table                   10000 non-null  object
 5   rate                         8628 non-null   object
 6   votes                        10000 non-null  int64 
 7   phone                        9820 non-null   object
 8   location                     9998 non-null   object
 9   rest_type                    9949 non-null   object
 10  dish_liked                   4450 non-null   object
 11  cuisines                     9990 non-null   object
 12  approx_cost(for two people)  9980 non-null   object
 13  reviews_list                 100

In [18]:
dataset.shape

(10000, 17)

In [19]:
dataset = dataset.drop(['url', 'address', 'phone', 'name', 'menu_item', 'dish_liked'], axis = 1)
dataset = dataset.dropna(axis = 0, how ='any')
dataset = dataset.rename(columns = {'approx_cost(for two people)' : 'cost', 'listed_in(type)' : 'type', 'listed_in(city)' : 'city'})

dataset['cost'] = dataset['cost'].astype(str)
dataset['cost'] = dataset['cost'].apply(lambda x: x.replace(',','.'))
dataset['cost'] = dataset['cost'].astype(float)

dataset_test = dataset.loc[dataset.rate == 'NEW']
dataset = dataset.loc[dataset.rate != 'NEW'].reset_index(drop = True)
dataset = dataset.loc[dataset.rate != '-'].reset_index(drop = True)
remove_slash = lambda x: x.replace('/5', '') if type(x) == np.str else x
dataset.rate = dataset.rate.apply(remove_slash).str.strip().astype('float')
dataset['rate'] = round(dataset['rate'])

dataset['online_order'].replace('Yes', 1,inplace = True)
dataset['online_order'].replace('No', 0,inplace = True)
dataset_test['online_order'].replace('Yes', 1,inplace = True)
dataset_test['online_order'].replace('No', 0,inplace = True)
dataset['book_table'].replace('Yes', 1,inplace = True)
dataset['book_table'].replace('No', 0,inplace = True)
dataset_test['book_table'].replace('Yes', 1,inplace = True)
dataset_test['book_table'].replace('No', 0,inplace = True)

dataset.info()
dataset_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8021 entries, 0 to 8020
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   online_order  8021 non-null   int64  
 1   book_table    8021 non-null   int64  
 2   rate          8021 non-null   float64
 3   votes         8021 non-null   int64  
 4   location      8021 non-null   object 
 5   rest_type     8021 non-null   object 
 6   cuisines      8021 non-null   object 
 7   cost          8021 non-null   float64
 8   reviews_list  8021 non-null   object 
 9   type          8021 non-null   object 
 10  city          8021 non-null   object 
dtypes: float64(2), int64(3), object(6)
memory usage: 689.4+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 542 entries, 72 to 9996
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   online_order  542 non-null    int64  
 1   book_table    542 non-null   

In [20]:
# NLP for cleaning and rating predictions
def cleaning(s):
    s = str(s)
    s = s.lower()
    s = re.sub('\s\W',' ',s)
    s = re.sub('\W,\s',' ',s)
    s = re.sub(r'[^\w]', ' ', s)
    s = re.sub("\d+", "", s)
    s = re.sub('\s+',' ',s)
    s = re.sub('[!@#$_]', '', s)
    s = s.replace("co","")
    s = s.replace("https","")
    s = s.replace(",","")
    s = s.replace("[\w*"," ")
    s = s.replace("rated rated n","")
    return s

dataset['reviews_list'] = [cleaning(s) for s in dataset['reviews_list']]
dataset_test['reviews_list'] = [cleaning(s) for s in dataset_test['reviews_list']]
dataset = dataset.loc[dataset.reviews_list != ' '].reset_index(drop = True)
dataset_test = dataset_test.loc[dataset_test.reviews_list != ' '].reset_index(drop = True)

In [21]:
def catagory_encoding(data):
    for i in range (0, len(dataset[data].unique())):
        dataset[data].replace(dataset[data].unique()[i], i, inplace = True)
    for i in range (0, len(dataset_test[data].unique())):
        dataset_test[data].replace(dataset_test[data].unique()[i], i, inplace = True)
    return

catagory_encoding('type')
catagory_encoding('cuisines')
catagory_encoding('location')
catagory_encoding('city')
catagory_encoding('rest_type')

dataset_test = dataset_test.drop(['rate'], axis = 1)

above function converts catagorial data into numeric and stores in same column

In [22]:
# Function for emotional analysis per gender
dataset_temp = dataset.copy()

def emotional_analysis(text):
    tokenize_words = word_tokenize(text)
    clean_words=[]
    for i in tokenize_words:
        if i not in stopwords.words("english"):
            clean_words.append(i)
    emotions = []
    with open("/content/drive/MyDrive/Restaurant /emotion.txt","r") as file:
        for i in file:
            temp = i.replace("\n","")
            temp = temp.strip()
            temp = temp.replace(" ","")
            temp = temp.replace(",","")
            temp = temp.replace("'","")
            word, emotion = temp.split(":")
            if word in clean_words:
                emotions.append(emotion)
    return emotions
        
def emotional_plotting(i):
    temp = ''
    for j in text[i - 1]:
        temp = temp + j
    emotion = emotional_analysis(temp)
    title = 'Emotion in review rated - ' + str(i)
    plotting_sns(emotion, title)
    return

text = [[],[],[],[],[]]
for rating in range (1, 6):
    for i in dataset['reviews_list'].loc[dataset['rate'] == rating]:
        text[rating - 1].append(i)

In [23]:
# Feature Engineering
count_vectorizer = CountVectorizer(max_features = 1000, stop_words = "english")

sparce_matrix = count_vectorizer.fit_transform(dataset_test['reviews_list']).toarray()
dataset_sparce_matrix = pd.DataFrame(data = sparce_matrix)
dataset_test = dataset_test.drop(['reviews_list'], axis = 1)
dataset_test = pd.concat([dataset_test, dataset_sparce_matrix], axis=1)

sparce_matrix = count_vectorizer.fit_transform(dataset['reviews_list']).toarray()
dataset_sparce_matrix = pd.DataFrame(data = sparce_matrix)
dataset = dataset.drop(['reviews_list'], axis = 1)
dataset = pd.concat([dataset, dataset_sparce_matrix], axis=1)

In [24]:
dataset.head()

,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,type,city,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,1,1,4.0,775,0,0,0,800.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,4,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,...,0,0,1,1,0,0,2,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,0,21,0,0,0,0,0,0,1,0,0
1,1,0,4.0,787,0,0,1,800.0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,0
2,1,0,4.0,918,0,1,2,800.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,7,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,14,0,1,1,0,0,0,1,0,0
3,0,0,4.0,88,0,2,3,300.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,3,0,0,0,0,0,0,0,1,0,0,0,0,0,5,10,...,5,6,0,0,0,0,2,0,0,0,5,0,0,1,1,0,0,0,0,0,0,0,4,0,0,0,0,0,2,0,21,0,0,6,0,0,0,0,0,0
4,0,0,4.0,166,1,0,4,600.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
# get the dataset
def get_dataset(dataset_y, dataset_x):
    y = dataset_y.values
    X = dataset_x
    return X, y

# get a stacking ensemble of models
def get_stacking():
    # define the base models
    level0 = list()
    level0.append(('Logistic Regression', LogisticRegression()))
    level0.append(('K Nearest Neighbour', KNeighborsClassifier()))
    level0.append(('Decision Tree Classifier', DecisionTreeClassifier()))
    level0.append(('Support Vector Classifier', SVC()))
    level0.append(('Gaussian Navy Bayse', GaussianNB()))
    level0.append(('ADA boost', AdaBoostClassifier()))
    level0.append(('Bagging Classifier', BaggingClassifier(n_estimators = 10)))
    level0.append(('Random Forest Classifier', RandomForestClassifier(n_estimators = 10)))
    level0.append(('Extra Trees Classifier', ExtraTreesClassifier(n_estimators = 10)))
    # define meta learner model
    level1 = LogisticRegression()
    # define the stacking ensemble
    model = StackingClassifier(estimators = level0, final_estimator = level1, cv = 5)
    return model

# get a list of models to evaluate
def get_models():
    models = dict()
    models['Logistic Regression'] = LogisticRegression()
    models['K Nearest Neighbour'] = KNeighborsClassifier()
    models['Decision Tree Classifier'] = DecisionTreeClassifier()
    models['Support Vector Classifier'] = SVC()
    models['Gaussian Navy Bayse'] = GaussianNB()
    models['ADA boost'] = AdaBoostClassifier()
    models['Bagging Classifier'] = BaggingClassifier()
    models['Random Forest Classifier'] = RandomForestClassifier()
    models['Extra Trees Classifier'] = ExtraTreesClassifier()
    models['Stacking'] = get_stacking()
    return models

# evaluate a give model using cross-validation
def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 3, random_state = 1)
    scores = cross_val_score(model, X, y, scoring = 'accuracy', cv = cv, n_jobs = -1, error_score = 'raise')
    return scores

In [27]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [28]:
# get database and plot the result of all models & predicting on new dataset
%%time
X, y = get_dataset(dataset['rate'], pd.get_dummies(dataset.drop('rate', 1)))
models = get_models()

results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, X, y)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
plt.boxplot(results, labels = names, showmeans = True)
plt.xticks(rotation = 90)
plt.show()

model_test = get_stacking()
model_test.fit(X, y)
yhat = model_test.predict(dataset_test)
dataset_test['rate'] = yhat

>Logistic Regression 0.766 (0.011)
>K Nearest Neighbour 0.774 (0.014)
>Decision Tree Classifier 0.885 (0.010)
>Support Vector Classifier 0.712 (0.002)
>Gaussian Navy Bayse 0.385 (0.010)
>ADA boost 0.724 (0.049)
>Bagging Classifier 0.913 (0.008)
>Random Forest Classifier 0.923 (0.009)
>Extra Trees Classifier 0.911 (0.010)


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


KeyboardInterrupt: ignored